In [ ]:
%matplotlib inline
import os
import math
import pandas as pd
import numpy as np
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, LabelBinarizer, Imputer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, AdaBoostClassifier

TITANIC_PATH = "/Users/JuliusHigiro/Machine-Learning/titanic/datasets/"

def load_titanic_data(dataset_path=TITANIC_PATH):
    csv_training_path = os.path.join(dataset_path, "train.csv")
    csv_test_path = os.path.join(dataset_path, "test.csv")
    return pd.read_csv(csv_training_path), pd.read_csv(csv_test_path)
            
def prefix(name):
    return name[name.find(","):name.find(".")].split(",")[1].strip()

def age_range(age):
    if age > 60: return 'elderly'
    elif 39 < age <= 60: return 'adult'
    elif 18 < age <= 39: return 'young adult'
    elif age <= 18: return 'child'
    else: return 'unknown' 

# A. INTRODUCTION

1. **Objective:** Predict the survival rate of passengers on the Titanic with the highest accuracy.
<br>
<br>
2. **Method:** 
    1. Explore the data and obtain insights and useful information for developing prediction models.
    2. Use information learned from the data exploration to create useful features that will enhance our models.
    3. Process and prepare the data for the various machine learning models that we explored in the project.
    4. Learn a prediction model on the dataset using several different classifiers.
<br>
<br>
3. **Data:** The dataset consists of 891 entries with some entries missing feature values.

    1. survival
    2. pclass
    3. sex
    4. Age
    5. sibsp
    6. parch
    7. ticket
    8. fare
    9. cabin
    10. embarked
<br>
<br>
4. **Results:** Our highest submission score was 0.78947.

In [ ]:
 training_data, test_data = load_titanic_data()

# B. EXPLORATORY DATA ANALYSIS

In [ ]:
# visualize the different predictors vs the target values

fig, axes = subplots(3)
fig.set_size_inches(10, 25)
j = 0;

predictors = array(['Pclass', 'SibSp', 'Parch'])

for predictor in predictors:
    values = unique(df[predictor])
    survived = numpy.zeros(len(values))
    totals = numpy.zeros(len(values))
    
    
    for i in range(0, len(values)):
        survived[i] = sum(training_data['Survived'][training_data[predictor] == values[i]] == 1)
        totals[i] = len(training_data['Survived'][training_data[predictor] == values[i]])
    sca(axes[j])
    
    bar(arange(len(values)), survived/totals)
    xticks(arange(len(values)), values)
    
    axes[j].set_title( 'Survival Rate by ' + predictor)
    
    j+= 1
    
    

The graph titled 'Survival Rate by Pclass' shows us the percentage of each of the different Pclasses that survived. It is clear that a passenger's Pclass impacted their survival rate. Pclass 1 had the highest, followed by Pclass 2, and then Pclass 3, suggesting that passengers with higher socio-economic status had a better chance of survival.
   

Subgraphs relating to SibSp and Parch also show that having a smaller family unit improved survival. If a passenger traveled with more than four childern and parents, then their chances of survival dropped significantly. This might suggest that passengers with many children or parents gave up their seat on a life boat for their chilren/parents. The same results are reflected in the SibSp chart; traveling with less than two siblings and spouses greatly increased your chance of survival.

In [ ]:
# a look at the embarked data
sns.factorplot(x="Embarked", kind="count", size=8, data=training_data)
sns.factorplot(x="Embarked", hue="Survived", col="Survived", kind="count", size=8, data=training_data)

By examining the 'Embarked' variable we can see that most of the passengers boarded at Southampton. Since more people boarded at that location, it isn't surprising that it had the greatest number of survivors. When we examine the three locations by survival percentage (See below), it is clear that Cherbourg had the highest percentage of survivors. This could be explained by the class of the guest that boarded at each location.

In [ ]:
sns.factorplot(x="Embarked", y="Survived", kind="bar", size=8, data=training_data)


The graph below shows the distribution of Pclass that boarded at each location. As we guessed, most of the passengers that boarded at Cherbourg belonged to a higher socio-economic class, specifically Pclass 1. We already know that Pclass 1 has the highest survival rate among the different Pclasses, so this helps to explain why Cherbourg had such a high survival percentage.

In [ ]:
sns.factorplot(x="Embarked", hue = 'Pclass', kind="count", size=8, data=training_data)

In [ ]:
Below we can see graphs that display both the total number that survived and the survival percentage categorized by sex. It is clear there were more female passengers that survived and that the percentage of females that survived is much higher than the percentage of males that survived.

In [ ]:
sns.factorplot(x="Sex", hue="Survived", col="Survived", kind="count", size=8, data=training_data)
sns.factorplot(x="Sex", y="Survived", kind="bar", size=6, data=training_data)


We can also examine the survival rate of the sexes by their Pclass. As expected, females have a higher rate of surival than the males in the same Pclass. It's interesting that see that a female in Pclass 3 has a better change of surival than a male in Pclass 1.

In [ ]:
sns.factorplot(x="Sex", y="Survived", hue="Pclass", kind="bar", size=8, data=training_data)

We will now take a look at the survival rate based on the age of the passenger.For this we group each passenger into one of five age groups - child, young adult, adult, elderly, or unknown. If a passenger is missing their age value we put them in the unknown group for now. These age groups are used only for the visualizations, not as actual predictors. In our model we use the exact age of each passenger as the predictor.

In [ ]:
training_data['Age_Groups'] = training_data.Age.map(age_range)
sns.factorplot(x="Age_Groups", hue="Survived", kind="count", size = 8, data=training_data)

sns.factorplot(x="Age_Groups", y="Survived", kind="bar", size = 8, data=training_data)

sns.factorplot(x="Age_Groups", hue="Survived", col="Sex", kind="count", aspect=0.5, size = 8, data=training_data)

As you can see above, children had the highest survival rate among all age groups. When you consider gender as well you see that more females survived than males in each age group. It seems the saying of "women and children first" holds true for the titanic. 

In [ ]:
sns.factorplot(x="Age_Groups", y="Survived", col="Pclass", kind="bar", aspect=0.5, size = 8, data=training_data)

If we examine each age group by their Pclass, we see that higher socio-economic groups tend to have higher survival rates than their counterpart in lower socio-economic groups. This reinforces our above findings. We also note that children have the highest survival rate out of every Pclass group. 

Finally, we examine the survival rate by Fare.

In [ ]:
sns.factorplot(x="Survived", y="Fare", hue="Sex", col="Sex", kind="bar", size = 8, data=training_data)
sns.factorplot(x="Pclass", y="Fare", col = "Survived", kind="box", size = 8, data=training_data)

It's interesting to note that passengers that paid a higher fare amount tended to have a higher rate of survival. This could have to do with the 'Cabin' group. Customers that payed more for their fare might have ended up in cabins closer to the stairs or to the life boats. This could help them get to the boats faster, which would increase their survival rate.

# C. FEATURE EXTRACTION

### Title

Now that we have read in our data, we can begin to consider some new features. At first glance the 'Name' column seems useless; we can't just use it as a predictor since almost every passenger will have a unique name. However we can extract the title from each passenger and add this into our models. This might give us even more ensight into the socio-economic status of a passenger.

In [ ]:
training_data["Title"] = training_data.Name.map(prefix)

In [ ]:
sns.factorplot(x="Title", y="Survived", kind="bar", size=8, aspect = 2, data=training_data)

As you can see above, classes that are associated with higher socio-economic status, like Lady, Sir, or  the Countess, have a higher survival rate. This graph also reinforces the data showing that females are more likely to survive than males. Titles associated with women, like Mrs, and Ms, have a much higher survival rate than the male counterpart Mr and Master. This graph can may be misleading since some titles are associated with very few passengers. To avoid overfitting we will group upper socio-economic male titles into a group called 'sir' and the upper socio-economic females titles into a group called 'Lady'.

In [ ]:
name_dict = {
                "Don":        "Sir",
                "Sir":        "Sir",
                "Col":        "Sir",
                "Dr":         "Sir",
                "Capt":       "Sir",
                "Major":      "sir",
                "Rev":        "Sir",
                "Jonkheer":   "Sir",
                "Lady" :      "Lady",
                "the Countess":"Lady",
                "Mrs" :       "Lady",
                "Mme":        "Lady",
                "Mlle":       "Lady",
                "Ms":         "Lady",
                "Mr" :        "Mr",
                "Miss" :      "Miss",
                "Master" :    "Master",
                "Lady" :      "Lady"

                }

In [ ]:
training_data['Title'] = training_data.Title.map(name_dict)
training_data['Title'].unique()

Below is a chart that shows the survival rate of the new groups. We can see that the 'Lady' group has the highest surival rate, which is to be expected. We also see that the 'Sir' group has a higher survival rate than the 'Mr' group. This is also expected since groups with a higher socio-economic status generally have a higher survival rate than other groups of the same gender.

In [ ]:

sns.factorplot(x="Title", y="Survived", kind="bar", size=8, data=training_data)


### Family Size

We thought it might also be worthwhile to examine the size of the family each passenger was traveling with. This can be accomplished by adding the number of siblings and spouses a passenger is traveling with (SibSp) and the number of parents and children a passenger is traveling with (Parch). From the results above we expect that smaller families units will have a higher rate of survival.

In [ ]:
training_data["Family_size"] = df.SibSp + df.Parch + 1

The graph below displays the rate of survival for the family size each passenger is traveling with. As expected smaller families have a higher survival rate.

In [ ]:
sns.factorplot(x="Family_size", y="Survived", kind="bar", size=8, data=training_data)

# Preprocessing of Data

We decided to impute the missing values in our data. This allowed us to use passengers with missing values in our training set. Each section will discuess how we calculated the missing values for each category. After these values were calculated we converted each categorical variable, such as Title or Pclass, into several that use one hot encoding. 

In [ ]:
lb = LabelBinarizer()
scaler = StandardScaler()
imputer = Imputer(strategy="median")

### Age

We noticed that there were several passengers that were missing their age. To compute this we will use the median age of the Pclass and Title groups that the passenger belongs to. Below we can see that each Title group has a different age associated with the differnt Pclasses. By including multiple levels in this fashion, we should get a more realistic estimate on the passengers' age.

In [ ]:
training_data.groupby(['Pclass','Title'])['Age'].median()

In [ ]:
training_data['Age'] = training_data.groupby(['Pclass','Title'])['Age'].transform(lambda x:x.fillna(x.median()))

### Embarked

Two passengers were missing the embarkment location. Their infomation is shown below:

In [ ]:
training_data[training_data.Embarked.isnull()]

We can see that both the passengers were female and belonged to Pclass 1. Below is a graph that shows the distribution of where females in Pclass 1 boarded. Since most of them boarded at Southampton, we will use it as the embarked location for the two passengers.

In [ ]:
sns.factorplot(x="Embarked", kind="count", data=training_data[(training_data['Sex'] == 'female') & (df['Pclass'] == 1)], size=8)

In [ ]:
embark_filler = 'S'
training_data.loc[(training_data.Embarked.isnull()), 'Embarked'] = embark_filler

embark = lb.fit_transform(training_data['Embarked'])
embark_columns = lb.classes_
embarked = pd.DataFrame(data=embark, columns=embark_columns)

### Title

There were now missing values in Title.

In [ ]:
title = lb.fit_transform(training_data['Title'])
title_columns = lb.classes_
titles = pd.DataFrame(data=title, columns=title_columns)

### Pclass

There were no missing values in Pclass.

In [ ]:
pclass = lb.fit_transform(training_data['Pclass'])
pclass_columns = ['Class1', 'Class2', 'Class3']
pclasses = pd.DataFrame(data=pclass, columns=pclass_columns)

### Sex

There were no missing values in Sex.

In [ ]:
sex = lb.fit_transform(training_data['Sex'])
genders = pd.DataFrame(data=sex, columns=['Sex_transform'])

### Combine Transformed Dataframes into Training Dataset

Now we combine all of the one hot encoded dataframes we created above to use as testing data. We also normalize our data in this step. This allows the unit changes for each predictor to have a more equal impact on our models. If we didn't normalize the data then changes in large integers, like age, might have more of an impact than changes in smaller integers in our data, like SibSp.

In [ ]:
numerical_attributes=['SibSp', 'Parch', 'Age', 'Family_size', 'Fare']
data = pd.concat([training_data[numerical_attributes], embarked, titles, pclasses, genders], axis=1)
numerical_attributes=['SibSp', 'Parch', 'Age', 'Family_size', 'Fare']
# scale the numerical attributes
data[numerical_attributes] = scaler.fit_transform(data[numerical_attributes])
targets = training_data['Survived']

# Model Building

The first models we choose to explore were based on our findings in Orange. SVC, Random Forest, and Adaboosting had the highest accuracy rate, so these were the first models we explored. Since the classification rate wasn't satisfactory we ended up exploring more models. Each section below discusses the different models and how we tuned the parameters. A seed was set for testing purposes.

In [ ]:
seed = 12345

### SVC Model

The first model we will build will be a support vector classifier. For this model we will try to tune gamma, which is the kernel coeffecient, and C, which is the penalty parameter for the error term. When we were first tuning the gamma value we tested different magnitude values (now commented out) to find the region we should be searching. This resulted in the gamma range you see below. To find the region for the penalty parameter we followed the same procedure. We also decided to hold back to testing set to see the approximate accuracy of our model. 

In [ ]:
# determine the best SVC model

X_train, X_test, y_train, y_test = train_test_split(
    data, targets, test_size=0.25, random_state=0)

tuned_parameters = [{'kernel': ['rbf'], 
                    'gamma': np.arange(1e-4, 1e-2, 1e-3),
                     'C': [150, 175, 180, 200]}]

clf = GridSearchCV(SVC(), tuned_parameters, cv=10,
                       scoring='accuracy')
clf.fit(X_train, y_train)

print('Best parameters for our model: ')
print(clf.best_params_)

y_true, y_pred = y_test, clf.predict(X_test)
print('\nClassification Report for SVC Model with above parameters:')
print(classification_report(y_true, y_pred))


The output above shows that our model is about 81% accurate. 

### Random Forest Model

For our random forest classifier we will be tuning n_estimators, which is the number of trees built, the criterion, the maximum features considered, the maximum depth our of trees, and whether or not the data is bootstraped. We followed the same procedure as the SVC model to determine the ranges we were searching for our parameter tuning. Again, we left out some test data to gauge the approximate accuracy of our model.

In [ ]:
# determine the best RandomForest model

X_train, X_test, y_train, y_test = train_test_split(
    data, targets, test_size=0.5, random_state=0)

tuned_parameters = [{'n_estimators': [300, 400], 
                     'criterion': ['gini', 'entropy'],
                     'max_features': ['auto', 'sqrt', 'log2'],
                     'max_depth': [5, 7],
                     'bootstrap': [True, False]}]

clf = GridSearchCV(RandomForestClassifier(), tuned_parameters, cv=10,
                       scoring='accuracy')

clf.fit(X_train, y_train)

print('Best parameters for our model: ')
print(clf.best_params_)

y_true, y_pred = y_test, clf.predict(X_test)
print('\nClassification Report for Random Forest Model with above parameters:')
print(classification_report(y_true, y_pred))

The output above shows that our model is about 82% accurate. With random forest classifiers we are also able to show the importance of our features in building the model.

In [ ]:
clf = RandomForestClassifier(n_estimators = 300, bootstrap = False, max_depth = 5)
clf.fit(data, targets)

importances = clf.feature_importances_
features = np.array(['SibSp', 'Parch', 'Age', 'Family_size', 'Fare', 'C', 'Q', 'S', 'Lady', 'Master', 'Miss', 'Mr',
                     'Mrs', 'Sir', 'Class1', 'Class2', 'Class3', 'Sex_transform'])
indices = np.argsort(importances)

plt.title('Feature Importances')
plt.barh(range(len(indices)), importances[indices], color='b', align='center')
plt.yticks(range(len(indices)), features) ## removed [indices]
plt.xlabel('Relative Importance')
plt.show()

As we can see above, Sex seems to be the most important feature, followed by the Pclass of the passenger. This suggests that gender has the biggest influence on who survives. This reinforces the graphs above that show females survive more often than males, regardless of the passenger's Pclass.

### Adaboosting Model

Another model we choose to explore was the adaboosting model. For this model we choose to tune the number of estimators, the learning rate, and the algoirthm used by the model. We set aside some test data to give the approximate accuracy of the tuned model.

In [ ]:
#determine the best Adaboost model

X_train, X_test, y_train, y_test = train_test_split(
    data, targets, test_size=0.5, random_state=0)

tuned_parameters = [{'n_estimators': [50, 100, 150], 
                     'learning_rate': [1, .5, .25, .1],
                     'algorithm': ['SAMME', 'SAMME.R']}]

clf = GridSearchCV(AdaBoostClassifier(), tuned_parameters, cv=10,
                       scoring='accuracy')

clf.fit(X_train, y_train)

print('Best parameters for our model: ')
print(clf.best_params_)

y_true, y_pred = y_test, clf.predict(X_test)
print('\nClassification Report for Random Forest Model with above parameters:')
print(classification_report(y_true, y_pred))

The classification report above shows that our model is approximately 79% accurate.

### Gradient Boosting Classifier

We also fit a gradient boosting classifier. For this model we tuned the number of boosting stages to perform, the maximum depth of the regression estimators, the minimum number of samples per leaf, the maximum number of features used when considering a split, and the learning rate. Again we set back some data to calculate the approximate accuracy of the model.

In [ ]:
# Tune parameters and fit Gradient Boosting Classifier to identify the best parameters

X_train, X_test, y_train, y_test = train_test_split(data, targets, test_size=0.25, random_state=seed)

parameters = {
   'n_estimators': [1000, 1100],
    'max_depth': [4, 6, 8, 10, 12],
    'min_samples_leaf': [5, 11, 17, 21],
    'max_features': ['auto', 'sqrt', 'log2'],
    'learning_rate': [0.001, 0.01, 0.1]   
}

gbc = GradientBoostingClassifier(random_state=seed)
gbc_gscv = GridSearchCV(estimator=gbc, cv=10, param_grid=parameters, scoring='accuracy').fit(X_train, y_train)

print('Best parameters for our model: ')
print(gbc_gscv.best_params_)

y_true, y_pred = y_test, gbc_gscv.predict(X_test)
print('\nClassification Report for Gradient Boosting Classifier Model with above parameters:')
print(classification_report(y_true, y_pred))

The classification report above shows that our model is approximately 79% accurate.

In [ ]:
params = {'n_estimators': 1000, 'max_depth': 6,
          'min_samples_leaf': 5, 'learning_rate': 0.001,
          'max_features': 'sqrt', 'random_state': seed}

gbcm = GradientBoostingClassifier(**params).fit(data, targets)

importances = gbcm.feature_importances_
features = np.array(['SibSp','Parch','Fare', 'Age',
              'Family_Size', 'Sex_transform',
              'Class1','Class2','Class3',
              'C','Q','S','Lady','Master','Miss',
              'Mr','Mrs','Sir'])

indices = np.argsort(importances)

plt.title('Feature Importances')
plt.barh(range(len(indices)), importances[indices], color='b', align='center')
plt.yticks(range(len(indices)), features) 
plt.xlabel('Relative Importance')
plt.show()

For this model the 'Title' group was the most important feature when fitting the model.

### DNN Classifier

We will also fit a DNN classifier. For this problem we will be tuning the number of nodes in each hidden layer. We chose to stick with two hidden layers. After experimenting with the magnitude of the number of nodes in each layer (3, 5, 10, 50, 100, 1000), we found that our accuracy was highest when we selected a number that is smaller than the number of predictors in our model. With that in mind we tuned our model by testing every combination of the number of nodes from one to the max number of predictors for each layer. We calculated the accuracy by setting some data back and using it as a validation set. After the model was fit we predicted values for the validation set and compared the predictions to the true value.

In [ ]:
X_train = data.iloc[:600, :].astype(np.float32)
y_train = targets.iloc[:600].astype(np.float32)
X_test = data.iloc[600:, :].astype(np.float32)
y_test = targets.iloc[600:].astype(np.float32)

feature_columns = [tf.feature_column.numeric_column('x', shape=[18])]

accuracies = np.zeros((17, 17))


for i in range(17):
    for j in range(17):
        
        classifier = tf.estimator.DNNClassifier(feature_columns=feature_columns,
                                                  hidden_units=[i+16, j+16],
                                                  n_classes=2,
                                                  optimizer=tf.train.AdamOptimizer())

        train_input_fn = tf.estimator.inputs.numpy_input_fn(
              x={"x": np.array(X_train)},
              y=np.array(y_train),
              num_epochs=None,
              shuffle=True)

        classifier.train(input_fn=train_input_fn, steps=5000)

        test_input_fn = tf.estimator.inputs.numpy_input_fn(
              x={"x": np.array(X_test)},
              y=np.array(y_test),
             num_epochs=1,
              shuffle=False)

        accuracy_score = classifier.evaluate(input_fn=test_input_fn)["accuracy"]
        accuracies[i, j] = accuracy_score

print("Finished")

Below is a heat map of the approximate accuracy of the different combinations of nodes per layer. You can see that most values are cloase to .8. The highest accuracy rate (.85) seems to be at 3 nodes for the first layer and 2 nodes in the second layer.

In [ ]:
accuracies = pd.DataFrame(accuracies)

plt.figure(figsize = (10,10))
ax = sns.heatmap(data=accuracies, cmap="YlGnBu", annot=True, linewidths=.5, cbar_kws={'label': 'Accuracy on Test Set'})
ax.set(xlabel='# of 2nd hidden layer nodes', ylabel='# of 1st hidden layer nodes')
ax.set_xticklabels(np.arange(1, 18))
ax.set_yticklabels(reversed(np.arange(1, 18)))

# Ensemble Methods

Since we have created so many models, we thought it would be worth looking into ways to combine the best models. Below is a graph that shows the approximate accuracy of each model. The three models with the highest accuracy rates are RFC, SVC, and GBC. We will use these three models going forward.

In [ ]:
class_rates = np.array([.81, .82, .79, .79, .85])
features = np.array(['SVC', 'RFC', 'AdaBoost', 'GBC', 'DNN'])

plt.title('Tuned Model Accuracy Rates')
plt.barh(range(len(class_rates)), class_rates, color='b', align='center')
plt.yticks(range(len(class_rates)), features) 
plt.xlabel('Accuracy')
plt.show()

Now that we have selected the best models, we will create a 10 fold cross-validation set. This will allow us to fit our best models (with the parameters selected above) with 9 of the folds and make predictions about the remainig fold. We will cycle through all of the folds and store all of the predictions in an array called 'predictions'. These prediction will be used to begin exploring ways to combine all of our best models.

In [ ]:
feature_columns = [tf.feature_column.numeric_column('x', shape=[18])]
skf = StratifiedKFold(n_splits = 10, shuffle = True)
j = 0

#create a numpy array with the CV predictions for the training data
predictions = np.zeros((len(data), 5))

#fit all of the models with the training data and fill and array with the predictions for the test data
for train_index, test_index in skf.split(data, targets):
    X_train = data.iloc[train_index]
    X_test = data.iloc[test_index]
    y_train = targets.iloc[train_index]
    y_test = targets.iloc[test_index]

    rf = RandomForestClassifier(max_depth = 9, n_estimators = 300)
    rf.fit(X_train, y_train)

    sv = SVC(C=100, gamma = .005, kernel = 'rbf')
    sv.fit(X_train, y_train)

    ada = AdaBoostClassifier(algorithm = 'SAMME', learning_rate = .5, n_estimators = 100)
    ada.fit(X_train, y_train)
    
    params={'n_estimators': 1000, 'max_depth': 12,
            'min_samples_leaf': 5, 'learning_rate': 0.001,
            'max_features': 'sqrt', 'random_state': seed}
    
    gbcm = GradientBoostingClassifier(**params).fit(X_train, y_train)
    
    classifier = tf.estimator.DNNClassifier(feature_columns=feature_columns,
                                                  hidden_units=[12, 3],
                                                  n_classes=2,
                                                  optimizer=tf.train.AdamOptimizer())

    train_input_fn = tf.estimator.inputs.numpy_input_fn(
              x={"x": np.array(X_train)},
              y=np.array(y_train),
              num_epochs=None,
              shuffle=True)

    classifier.train(input_fn=train_input_fn, steps=5000)

    predict_input_fn = tf.estimator.inputs.numpy_input_fn(
            x={"x": np.array(X_test)},
            num_epochs=1,
            shuffle=False)

    kaanPredictions = list(classifier.predict(input_fn=predict_input_fn))
    kaanPredictions=np.array(kaanPredictions)
    finalPreds = []
    for i in range(len(kaanPredictions)):
        temp = kaanPredictions[i]['class_ids'][0]
        finalPreds.append(temp)


    rf_test = rf.predict(X_test)
    sv_test = sv.predict(X_test)
    ada_test = ada.predict(X_test)
    gbcm_test = gbcm.predict(X_test)

    y_true = y_test

    for k in range(0, len(test_index)):
        predictions[test_index[k]][0] = rf_test[k]
        predictions[test_index[k]][1] = sv_test[k]
        predictions[test_index[k]][2] = ada_test[k]
        predictions[test_index[k]][3] = gbcm_test[k]
        predictions[test_index[k]][4] = kaanPredictions[k]['class_ids'][0]
 


### Stacked Ensemble Learner

This learner will use logestic regression to figure out how to combine the models. This will allow the model to figure out where the different models work well, and where they are weakest. To tune the parameters we experimented with the penalty type and the regularization strength. We will use a 10 fold CV set to find the approximate accuracy of the model.

In [ ]:
#stacked ensemble model with best SV, RF, LOGREG, ADA, and DNN model
acc = np.zeros(10)
j = 0
for train_index, test_index in skf.split(predictions, targets):
    logreg_stack = linear_model.LogisticRegression(penalty = 'l2', C=1e2)
    logreg_stack.fit(predictions[train_index], targets[train_index])
    
    y_true, y_pred = targets[test_index], logreg_stack.predict(predictions[test_index])
    
    print('\nFold {} Classification Report'.format(j+1))
    print(classification_report(y_true, y_pred))
    acc[j] = 1.0 * sum(y_pred == y_true) / len(test_index)
    j += 1

print('\nAverage Accuracy of the model:')
print(mean(acc))

### 'Majority Vote' Ensemble

We will also consider a 'majority vote' ensemble. This learner will use the predictions of the best models to predict whether a passenger will surive or not. The model simply assigns each passenger the prediction the majority of the models have assigned it.

In [ ]:
sum(predictions[1])

In [ ]:
# majority vote ensemble with best models

maj = np.zeros(len(predictions))

for i in range(0, len(maj)):
    maj[i] = sum(predictions[i])
    
maj[maj < 3] = 0
maj[maj >= 3] = 1  

print('\nClassification Report')
print(classification_report(targets, maj))

# Preprocessing of Test Data

We will follow the same preprocessing procedures as the training data for the test data. 

In [ ]:
test_data.info()

### Fare

There was one missing values in Fare for the test data, so we used the median of all the Fares.

In [ ]:
test_fare = imputer.fit_transform(test_data[['Fare']])
fares_test = pd.DataFrame(test_fare, columns=['Fare'])

### Title

In [ ]:
test_data['Title'].unique()

We will follow the same basic procedure. This time there are significantly fewer Titles, so only a few need to be mapped to 'Sir' and 'Lady'.

In [ ]:
test_data['Title'] = test_data.Name.map(prefix)
test_data['Title'].unique()

In [ ]:

test_data.loc[(df_test.Title == 'Rev') |
            (df_test.Title == 'Dr') |
            (df_test.Title == 'Col'), 'Title']='Sir'

test_data.loc[(df_test.Title == 'Dona'), 'Title']='Lady'

title_test = lb.fit_transform(test_data['Title'])
title_columns_test = lb.classes_
titles_test = pd.DataFrame(data=title_test, columns=title_columns_test)

### Family Size

In [ ]:
test_data['Family_size'] = test_data.SibSp + test_data.Parch + 1 

### Pclass

In [ ]:
pclass_test = lb.fit_transform(test_data['Pclass'])
pclass_columns_test = ['Class1', 'Class2', 'Class3']
pclasses_test = pd.DataFrame(data=pclass_test, columns=pclass_columns_test)

### Sex

In [ ]:
sex_test = lb.fit_transform(test_data['Sex'])
genders_test = pd.DataFrame(data=sex_test, columns=['Sex_transform'])

### Embark

There are no missing 'Embarked' values in this data set, so we will just need to convert it to a one-hot encoded dataframe.

In [ ]:
embark_test = lb.fit_transform(test_data['Embarked'])
embark_columns = lb.classes_
embarked_test = pd.DataFrame(data=embark_test, columns=embark_columns)

### Age

The training and testing datasets were combined to compute the median age values grouped by class and title.

In [ ]:
test_data.groupby(['Pclass','Title'])['Age'].median()
training_data.groupby(['Pclass','Title'])['Age'].median()

In [ ]:
group_train_test_age = pd.concat([training_data[['Age','Pclass', 'Title']], test_data[['Age','Pclass', 'Title']]], ignore_index = True)


In [ ]:
group_train_test_age.groupby(['Pclass','Title'])['Age'].median()

In [ ]:
median_age = group_train_test_age.median()
test_data.loc[(test_data.Title == 'Ms'), 'Age'] = median_age[0]

In [ ]:
test_data['Age'] = group_train_test_age.groupby(['Pclass','Title'])['Age'].transform(lambda x:x.fillna(x.median()))

### Combine Transformed Dataframes into Testing Dataset

In [ ]:
numerical_attributes=['SibSp', 'Parch', 'Age', 'Family_size']
data_test = pd.concat([test_data[numerical_attributes], fares_test, embarked_test, titles_test, pclasses_test, genders_test], axis=1)
numerical_attributes=['SibSp', 'Parch', 'Age', 'Family_size', 'Fare']
# scale the numerical attributes
data_test[numerical_attributes] = scaler.fit_transform(data_test[numerical_attributes])

In [ ]:
data_test.info()

# Final Predictions

In [ ]:
data_test.drop('Ms', axis=1, inplace=True)

In [ ]:
# Fit the models
rf.fit(data, targets)

sv.fit(data, targets)

ada.fit(data, targets)
     
gbcm.fit(data, targets)
    
classifier = tf.estimator.DNNClassifier(feature_columns=feature_columns,
                                                  hidden_units=[12, 3],
                                                  n_classes=2,
                                                  optimizer=tf.train.AdamOptimizer())
    
train_input_fn = tf.estimator.inputs.numpy_input_fn(
              x={"x": np.array(data)},
              y=np.array(targets),
              num_epochs=None,
              shuffle=True)

classifier.train(input_fn=train_input_fn, steps=10000)


# Obtain predictions on the test data
rf_test = rf.predict(data_test)
sv_test = sv.predict(data_test)
ada_test = ada.predict(data_test)
gbcm_test = gbcm.predict(data_test)



predict_input_fn = tf.estimator.inputs.numpy_input_fn(
            x={"x": np.array(data_test)},
            num_epochs=1,
            shuffle=False)

dnnPredictions = list(classifier.predict(input_fn=predict_input_fn))
dnnPredictions=np.array(dnnPredictions)
finalDnnPreds = []
for i in range(len(dnnPredictions)):
    temp = dnnnPredictions[i]['class_ids'][0]
    finalDnnPreds.append(temp)

model_predictions = np.zeros((len(data_test), 5))

model_predictions[:,0] = rf_test
model_predictions[:,1] = sv_test
model_predictions[:,2] = gbcm_test
model_predictions[:,3] = ada_test
model_predictions[:,4] = finalDnnPreds

#use the stack_model fitted with all of the training data
logreg_final = linear_model.LogisticRegression(penalty = 'l2', C=1e2)
logreg_final.fit(predictions, targets)

test_preds = logreg_final.predict(model_predictions)

In [ ]:
#export the data
submission = pd.DataFrame({'PassengerId': df_test['PassengerId'] , 'Survived': test_preds})
submission.to_csv('titanic.csv', index=False)

# Conclusion

Our best model ended up with an accuracy of .78947. To improve this score even further we would devote some more time to feature extraction. In our model we didn't include the predictors 'Ticket' or 'Cabin'. When we examined Ticket, there were 681 unique values out of the 891 entries. Since the vast majority of passengers had a unique ticket number we didn't see any value in including it in our models. The 'Cabin' predictor could have been very useful. Passengers closer to stairs or the lifeboats might have an increased rate of survival. We were unable to use it as a predictor because of all of the missing values in the data. It might be worth while to see if we are able to use the information in the ticket column to assign each passenger to a cabin.
<br>
<br>
There were several challenges we encountered while working on this project. One particularly vexing challenge was handling the missing values in the data. The training set and test set were both missing large amounts of the 'Age' feature. We tried to impute these 'Age' values with a predictive neural net model trained on the other features, but problems with importing these values back into the original pandas DataFrame prevented us from fully realizing this imputation. We decided to instead impute 
these values with the median value of both the training and test set combined. 
<br>
<br>
Another challenge/limitation was the use of black-box API calls to run our learning algorithms. For instance, our neural net classifier was built using the tensorflow.estimator.DNNClassifier class, which automatically creates a neural net scoped with user parameters. This offered ease of implementation at the possible cost of greater control over the model. We encountered the same limitation when using the scikit-learn API to build our other models, although this was somewhat mitigated by our usage of hyperparameter optimization. We trained our neural net model 289 times with different combinations of hidden layer width to determine which would give us the best performance. We also used grid search to tune the parameters that we passed to our SVM, random forest, GBC, and Adaboosting classifiers. 
<br>
<br>
Finally, our team felt that the data set was rather limited in the sense that of the 2200 actual passengers on board, we only had 891 training values, with many missing feature values for age. We understand that data in real-world, practical situations often has similar limitations, but that does not make the task before us any easier. We also believe that leaving out crew member data (we think that's the case), leads to some loss in predictive power. 